In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPUs found. Training will default to CPU.")

No GPUs found. Training will default to CPU.


In [11]:
data_dir = 'data'

In [12]:
# Define parameters
img_width, img_height = 128, 128
batch_size = 32

In [13]:
# Create an ImageDataGenerator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.1765  # Approx 15% for validation, when combined with 70% training
)


In [14]:
# Create an ImageDataGenerator for testing (no augmentation, just scaling)
test_datagen = ImageDataGenerator(rescale=1./255)

In [15]:
# Prepare iterators
train_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'  # Set as training data
)


Found 9783 images belonging to 2 classes.


In [16]:
validation_generator = train_datagen.flow_from_directory(
    data_dir + '/train',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False
)


Found 2096 images belonging to 2 classes.


In [17]:
test_generator = test_datagen.flow_from_directory(
    data_dir + '/test',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)


Found 2000 images belonging to 2 classes.


In [40]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input
import keras

# Load a pre-trained ResNet and add custom layers
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_height, img_width, 3)))
base_model.trainable = False  # Freeze the base model

# OLD MODEL: model1.h5
model = keras.models.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D(2, 2),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
    
])

# NEW MODEL: model2.h5
# model = keras.models.Sequential([
#     base_model,  # Pre-trained ResNet50 base model
#     keras.layers.GlobalAveragePooling2D(),  # Reduce dimensionality and prevent overfitting
#     keras.layers.Dense(64, activation='relu'),  # First Dense layer
#     keras.layers.Dropout(0.5),  # Dropout to combat overfitting
#     keras.layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
# ])


# model5.h5
# model = keras.models.Sequential([
#     # Start with a simplified version of your original model
#     keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_height, img_width, 3)),
#     keras.layers.MaxPooling2D(2, 2),
#     keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
#     keras.layers.MaxPooling2D(2, 2),
#     keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
#     keras.layers.MaxPooling2D(2, 2),
    
#     # Global average pooling layer instead of flattening to reduce dimensionality
#     keras.layers.GlobalAveragePooling2D(),
    
#     # Dense layers for classification
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dropout(0.5),
#     keras.layers.Dense(1, activation='sigmoid')
# ])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 126, 126, 32)      896       
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 28800)             0         
                                                                 
 dense_16 (Dense)            (None, 64)               

In [46]:
# Run this cell to train model

import tensorflow as tf
import keras
import scipy

print("TensorFlow Version:", tf.__version__)
print("Keras Version:", keras.__version__)
print("Scipy Version:", scipy.__version__)

epochs = 15
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
)
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)
model.save('model5.h5')

TensorFlow Version: 2.15.0
Keras Version: 2.15.0
Scipy Version: 1.12.0
Epoch 1/15
305/305 [==============================] - 97s 318ms/step - loss: 0.3672 - accuracy: 0.8436 - val_loss: 0.4998 - val_accuracy: 0.7827
Epoch 2/15
305/305 [==============================] - 92s 301ms/step - loss: 0.3675 - accuracy: 0.8436 - val_loss: 0.5189 - val_accuracy: 0.7861
Epoch 3/15
305/305 [==============================] - 91s 299ms/step - loss: 0.3620 - accuracy: 0.8504 - val_loss: 0.5394 - val_accuracy: 0.7846
Epoch 4/15
305/305 [==============================] - 94s 306ms/step - loss: 0.3551 - accuracy: 0.8495 - val_loss: 0.4964 - val_accuracy: 0.7981
Epoch 5/15
305/305 [==============================] - 91s 299ms/step - loss: 0.3479 - accuracy: 0.8549 - val_loss: 0.4987 - val_accuracy: 0.7788
Epoch 6/15
305/305 [==============================] - 92s 302ms/step - loss: 0.3521 - accuracy: 0.8500 - val_loss: 0.5222 - val_accuracy: 0.7649
Epoch 7/15
63/63 [==============================] - 4s 58ms

/home/afriedsam/Programming/intro-AI/env/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [48]:
# RUN THIS CELL TO LOAD PRETRAINED MODEL

model = keras.models.load_model('model5.h5')

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy}')

63/63 [==============================] - 4s 68ms/step - loss: 0.3054 - accuracy: 0.8890
Test accuracy: 0.8889999985694885


In [61]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Generate predictions for the entire test set
predictions = model.predict(test_generator)

# Tweaking cutoff for prediction probability reduces false negatives by a lot but doesn't decrease accuracy too much
# 0.3:
    # Accuracy: 86%
    # Correct Predictions: 1725/2000
    # False Positives: 249/2000
    # False Negatives 26/2000
# 0.35
    # Accuracy: 88%
    # Correct Predictions: 1755/2000
    # False Positives: 203/2000
    # False Negatives 42/2000
# 0.4
    # Accuracy: 89%
    # Correct Predictions: 1771/2000
    # False Positives: 173/2000
    # False Negatives 56/2000
# 0.45
    # Accuracy: 89%
    # Correct Predictions: 1780/2000
    # False Positives: 141/2000
    # False Negatives 79/2000
# 0.5
    # Accuracy: 89%
    # Correct Predictions: 1778/2000
    # False Positives: 113/2000
    # False Negatives 109/2000
predicted_classes = (predictions > 0.5).astype(int)
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Compute confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)
true_positive = cm[1, 1]
false_positive = cm[0, 1]
true_negative = cm[0, 0]
false_negative = cm[1, 0]

# Display results
print("Confusion Matrix:")
print(cm)

print("\nMetrics:")
print(f"Number of Images: {len(true_classes)}")
print(f"Correctly Predicted: {true_positive + true_negative}")
print(f"False Positives: {false_positive}")
print(f"False Negatives: {false_negative}")
print(f"Accuracy: {round((true_positive + true_negative) / 2000 * 100)}%")

63/63 [==============================] - 5s 74ms/step
Confusion Matrix:
[[887 113]
 [109 891]]

Metrics:
Number of Images: 2000
Correctly Predicted: 1778
False Positives: 113
False Negatives: 109
Accuracy: 89%
